<a href="https://colab.research.google.com/github/lkolebka/Twitter-AI-detection-/blob/main/Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip uninstall tweepy

In [ ]:
import re 
import numpy as np 
import tweepy
from tweepy import OAuthHandler 
from textblob import TextBlob 
import matplotlib.pyplot as plt
import pandas as pd
from wordcloud import WordCloud
!pip install better-profanity

In [22]:
from better_profanity import profanity

In [1]:
exit ()


In [6]:
#Definining the access token for the App

consumer_key = ''
consumer_secret = '' 
access_token = ''
access_token_secret = ''

In [7]:
# Access Twitter Data

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [ ]:
# Input a query from the user

query = input("Please enter your topic / person of interest: ")

# Filter the query to remove retweets

filtered = query + "-filter:retweets"

# Generate the latest tweets on the given query 

tweets = tweepy.Cursor(api.search_tweets, 
                           q=filtered,
                           lang="en").items(100)

# Create a list of the tweets, the users, and their location

list1 = [[tweet.text, tweet.user.screen_name, tweet.user.location] for tweet in tweets]

In [115]:
# Convert the list into a dataframe

df = pd.DataFrame(data=list1, 
                    columns=['tweets','user', "location"])
# Convert only the tweets into a list

tweet_list = df.tweets.to_list()

In [ ]:
# raw Tweets list
tweet_list

In [117]:
# Create a function to clean the tweets. Remove profanity, unnecessary characters, spaces, and stopwords.

def clean_tweet(tweet):
    if type(tweet) == np.float:
        return ""
    r = tweet.lower()
    r = profanity.censor(r)
    r = re.sub("'", "", r) # This is to avoid removing contractions in english
    r = re.sub("@[A-Za-z0-9_]+","", r)
    r = re.sub("#[A-Za-z0-9_]+","", r)
    r = re.sub(r'http\S+', '', r)
    r = re.sub('[()!?]', ' ', r)
    r = re.sub('\[.*?\]',' ', r)
    r = re.sub("[^a-z0-9]"," ", r)
    r = r.split()
    stopwords = ["for", "on", "an", "a", "of", "and", "in", "the", "to", "from"]
    r = [w for w in r if not w in stopwords]
    r = " ".join(word for word in r)
    return r

In [ ]:
cleaned = [clean_tweet(tw) for tw in tweet_list]
cleaned

In [119]:
# Define the sentiment objects using TextBlob

sentiment_objects = [TextBlob(tweet) for tweet in cleaned]

sentiment_objects[0].polarity, sentiment_objects[0]

(0.0, TextBlob("no hes leaving because elon musk"))

In [ ]:
# Create a list of polarity values and tweet text

sentiment_values = [[tweet.sentiment.polarity, str(tweet)] for tweet in sentiment_objects]

# Print the value of the 0th row.

sentiment_values[0]

# Print all the sentiment values

sentiment_values[0:99]

In [ ]:
# Create a dataframe of each tweet against its polarity

sentiment_df = pd.DataFrame(sentiment_values, columns=["polarity", "tweet"])

sentiment_df

In [ ]:
#Custom  : create a dataframe of each tweet against its polarity and add the Result of the popularity

sentiment_df = pd.DataFrame(sentiment_values, columns=["polarity", "tweet",])


# Generate result using pandas
result = []
for value in sentiment_df["polarity"]:
    if value > 0:
        result.append("Positive")
    elif value < 0 : 
        result.append("Negative")
    else:
        result.append("Neutral")
      
sentiment_df["Result"] = result  
print(sentiment_df)

#specify the name of the file 
nameFile = "twitter.csv" 

#encoding of the file
sentiment_df.to_csv(nameFile, sep='\t', encoding='utf-8')

#download the file 
from google.colab import files
files.download(nameFile)



In [ ]:
# Save the tweet column as 't'.

t=sentiment_df['tweet']

# Save the polarity column as 'n'.

n=sentiment_df["polarity"]

# Convert this column into a series, 'm'. 

m=pd.Series(n)

m

In [ ]:
# Initialize variables, 'pos', 'neg', 'neu'.

pos=0
neg=0
neu=0

# Create a loop to classify the tweets as Positive, Negative, or Neutral.
# Count the number of each.


for items in m:


    if items>0:
        print("Positive"),
        pos=pos+1
    elif items<0:
        print("Negative")
        neg=neg+1
    else:
        print("Neutral")
        neu=neu+1
        
print("Positive : ", pos)
print("Negative : ", neg)
print("Neutre : ", neu)

print(my_loop)

In [ ]:
# Creation of a data shart
pieLabels=["Positive","Negative","Neutral"]

populationShare=[pos,neg,neu]

figureObject, axesObject = plt.subplots()

axesObject.pie(populationShare,labels=pieLabels,autopct='%1.2f',startangle=90)

axesObject.axis('equal')

plt.show()

In [ ]:
# Create a Wordcloud from the tweets

all_words = ' '.join([text for text in cleaned])
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()